In [3]:

from image_slicer import slice
from PIL import Image
print("imported slicer")
import numpy
print("imported numpy")
import cv2
print(cv2.__version__)
print("imported cv2")
file = "./image/a.png"
imgg = Image.open(file)
thresh = 200
fn = lambda x : 255 if x > thresh else 0
r = imgg.convert('L').point(fn, mode='1')
r.save('./image/aa.png')

img = cv2.imread("./image/aa.png")
cv2.imshow("source image", img)
cv2.waitKey(0)



imported slicer
imported numpy
4.5.1
imported cv2


-1

In [88]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os


wr, hr = 40, 10
imageArray = [[0 for x in range(wr)] for y in range(hr)]
cleanArray = [[0 for x in range(wr)] for y in range(hr)]

w=0
h=0
counter=0
key=[0, 0, 0]
file = "./image/test.png"
img = Image.open(file)
thresh = 200
fn = lambda x : 255 if x > thresh else 0
r = img.convert('L').point(fn, mode='1')
r.save(file)
imgRead = cv2.imread(file)
ar = np.array(imgRead)
for r in range(0,imgRead.shape[0],10):
    # print("petla pierwsza wykonalem sie h=",h," raz")
    for c in range(0,imgRead.shape[1],10):
        # print("petla druga wykonalem sie w=",w," raz")
        cv2.imwrite(f"./imageSplitted/img{r}_{c}.png",imgRead[r:r+10, c:c+10,:])
        image = Image.open(f"./imageSplitted/img{r}_{c}.png")
        imageArray[h][w]=np.array(image)
        if np.any(imageArray[h][w] == key) != True:
            cleanArray[h][w]=0
        else:
            counter+=1
            cleanArray[h][w]=1
        try: 
            os.remove(f"./imageSplitted/img{r}_{c}.png")
        except: pass   
        w+=1         
    h+=1
    w=0
for c in cleanArray:
    print(c)

[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [89]:

cleanArray
arr=np.sum(cleanArray,axis=0)
print()
print (arr)
sum=0
List= []
ListOfArrays= []
i=0
j=0
letterLength=0
for e in range (40):
    if arr[e] !=0:
        arr[e]=1
    
print (arr)
for ee in range (40):
    if arr[ee] != 0:
        if(arr[ee]*arr[ee+1]==0):
            letterLength+=1
            print("last",ee)
            List.append(ee)
        else:
            letterLength+=1
        print(letterLength)
    else:
        print(0)
        if(ee<39):
            if(arr[ee]+arr[ee+1]==1):
                print("first",ee)
                List.append(ee+1)
                letterLength=0

letters=(len(List))/2
print(letters)
print(List)   

for rows in range (10):  # outer loop  
        for columns in range (40):  # inner loop
            print(cleanArray[rows][columns], end = " ") # print the elements 
        print() 
wrr, hrr = 40, 10      
for xx in range (int(letters)):
    sthArray = [[0 for y in range(List[1+(2*xx)]-List[0+(2*xx)]+1)] for x in range(hrr)]
    # print("nowa iteracja")
    for rows in range (10):  # outer loop  
        for columns in range (40):  # inner loop
            if columns>= List[0+(2*xx)] and columns  <= List[1+(2*xx)]:
                sthArray[rows][columns-List[0+(2*xx)]]=cleanArray[rows][columns]
                # print("kolumna: ",columns)
                # print("wiersz: ",rows)
    for c in sthArray:
        print(c)
    ListOfArrays.append(sthArray)

    



        

    






[0 0 0 5 4 4 5 3 3 0 0 0 0 6 1 2 0 0 0 4 4 2 2 4 3 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
[0 0 0 1 1 1 1 1 1 0 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
0
0
0
first 2
1
2
3
4
5
last 8
6
0
0
0
0
first 12
1
2
last 15
3
0
0
0
first 18
1
2
3
4
5
6
last 25
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3.0
[3, 8, 13, 15, 19, 25]
0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [92]:
ListOfArrays[2]
    

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 0, 0],
 [1, 1, 0, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0]]